# L3: Image generation app 🎨

Load your HF API key and relevant Python libraries

In [ ]:
import os
import io
import IPython.display
from PIL import Image
import base64 
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
# run it locally
hf_api_key = os.environ['HF_API_KEY']
hf_inf_key = os.environ['HF_INF_KEY']

# Run it using inference API

In [ ]:
import requests

API_URL = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
headers = {"Authorization": f"Bearer {hf_inf_key}"}

payload = {
    "inputs": "a robot on Mars",
	"negative_prompt": "green grass",
	"num_inference_steps": 25,
	"guidance_scale": 7,
	"width": 512,
	"height": 512
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.content
image_bytes = query(payload)
# You can access the image with PIL.Image for example
image = Image.open(io.BytesIO(image_bytes))
image.show()

# Run it locally

In [ ]:
# Helper function
from diffusers import StableDiffusionPipeline
import torch
# this model also supports negative prompt steps guidance and width/height
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, use_safetensors=True).to("mps")
# Recommended if your computer has < 64 GB of RAM
# pipe.enable_attention_slicing()
prompt = "a photo of an astronaut on Mars"
image = pipe(prompt).images[0]
image.show()
image.save("output.png")    


## Building an image generation app 

Here we are going to run `runwayml/stable-diffusion-v1-5` using the `🧨 diffusers` library.

We can us both IPython.display.HTML or Image.open to display the output image.

In [ ]:
prompt = "a basketball lying on the playground"
image = pipe(prompt).images[0]
image.save("output.png") 



# IPython.display.HTML(f'<img src="data:image/png;base64,{base64.b64encode(open("output.png", "rb").read()).decode()}" />')
Image.open('output.png')


## Generating with `gr.Interface()`

### when the image is in the same directory

In [ ]:
def get_completion(prompt):
    image = pipe(prompt).images[0]
    image.save("output.png") 
    return "output.png"


get_completion("a man in a red shirt")

import gradio as gr

gr.close_all()
demo = gr.Interface(fn=get_completion,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result", type="filepath")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

demo.launch()   

In [ ]:
demo.close()

### When the image is in base64 format.

In [ ]:
import gradio as gr
#A helper function to convert the PIL image to base64
#so you can send it to the API
import requests

API_URL = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
headers = {"Authorization": f"Bearer {hf_inf_key}"}

def query(inputs):
    payload = {"inputs": inputs}
    response = requests.post(API_URL, headers=headers, json=payload)
    image = Image.open(io.BytesIO(response.content))
    image.save("output.jpg")
    img_base64 = base64.b64encode(open("output.jpg", "rb").read()).decode('utf-8')
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image



gr.close_all()
demo = gr.Interface(fn=query,
                    inputs=[gr.Textbox(label="Your prompt")],
                    outputs=[gr.Image(label="Result",type="pil")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never",
                    examples=["the spirit of a tamagotchi wandering in the city of Vienna","a mecha robot in a favela"])

demo.launch(share=True)

In [ ]:
demo.close()

## Building a more advanced interface

### when the image is in the same directory.
You can refer to below link to see how different parameters work in the pipeline.(so you can use those when running locally)   
[pipeline doc](https://huggingface.co/docs/diffusers/v0.30.0/en/api/pipelines/stable_diffusion/text2img#diffusers.StableDiffusionPipeline)

In [ ]:
import gradio as gr 

# the 
def get_completion(prompt, negative_prompt, steps, guidance, width, height):

    image = pipe(prompt=prompt,negative_prompt=negative_prompt, steps=steps, guidance=guidance, width=width, height=height).images[0]
    image.save("output.png") 
    return "output.png"


gr.close_all()
demo = gr.Interface(fn=get_completion,
                    inputs=[
                        gr.Textbox(label="Your prompt"),
                        gr.Textbox(label="Negative prompt"),
                        gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                 info="In how many steps will the denoiser denoise the image?"),
                        gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
                                  info="Controls how much the text prompt influences the result"),
                        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
                        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
                    ],
                    outputs=[gr.Image(label="Result", type="filepath")],
                    # the example should have as many parameters as above
                    examples=[["the spirit of a tamagotchi wandering in the city of Vienna","countryside",25,7,512,512],["a mecha robot in a favela","human",25,7,512,512]],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never"
                    )

demo.launch(share=True)



### When the image is in base64 format.
as the inference endpoint does not support other parameters other than inputs, so other parameters won't take effect.

In [ ]:
import gradio as gr
#A helper function to convert the PIL image to base64
#so you can send it to the API
import requests
import json

API_URL = "https://api-inference.huggingface.co/models/runwayml/stable-diffusion-v1-5"
headers = {"Authorization": f"Bearer {hf_inf_key}"}

def query(prompt, negative_prompt, steps, guidance, width, height):
    payload = {
        "inputs": prompt,
        "negative_prompt": negative_prompt,
        "steps": steps,
        "guidance": guidance,
        "width": width,
        "height": height}
    
    json_string= json.dumps(payload)

    
    response = requests.post(API_URL, headers=headers, json=payload)
    # print(response.content)
    image = Image.open(io.BytesIO(response.content))
    image.save("output.jpg")
    img_base64 = base64.b64encode(open("output.jpg", "rb").read()).decode('utf-8')
    base64_decoded = base64.b64decode(img_base64)
    byte_stream = io.BytesIO(base64_decoded)
    pil_image = Image.open(byte_stream)
    return pil_image


gr.close_all()
demo = gr.Interface(fn=query,
                    inputs=[
                        gr.Textbox(label="Your prompt"),
                        gr.Textbox(label="Negative prompt"),
                        gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                 info="In how many steps will the denoiser denoise the image?"),
                        gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
                                  info="Controls how much the text prompt influences the result"),
                        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
                        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
                    ],
                    outputs=[gr.Image(label="Result", type="filepath")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never")

demo.launch(share=True)

#### gr.Slider()
- You can set the `minimum`, `maximum`, and starting `value` for a `gr.Slider()`.
- If you want the slider to increment by integer values, you can set `step=1`.

In [ ]:
gr.close_all()
demo = gr.Interface(fn=get_completion,
                    inputs=[
                        gr.Textbox(label="Your prompt"),
                        gr.Textbox(label="Negative prompt"),
                        gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                                 info="In how many steps will the denoiser denoise the image?"),
                        gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7, 
                                  info="Controls how much the text prompt influences the result"),
                        gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512),
                        gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512),
                    ],
                    outputs=[gr.Image(label="Result")],
                    title="Image Generation with Stable Diffusion",
                    description="Generate any image with Stable Diffusion",
                    allow_flagging="never"
                    )

demo.launch(share=True)

In [ ]:
demo.close()

## `gr.Blocks()`

- Within `gr.Blocks()`, you can define multiple `gr.Row()`s, or multiple `gr.Column()`s.
- Note that if the jupyter notebook is very narrow, the layout may change to better display the objects.  If you define two columns but don't see the two columns in the app, try expanding the width of your web browser, and the screen containing this jupyter notebook.

- When using `gr.Blocks()`, you'll need to explicitly define the "Submit" button using `gr.Button()`, whereas the 'Clear' and 'Submit' buttons are automatically added when using `gr.Interface()`.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Image Generation with Stable Diffusion")
    prompt = gr.Textbox(label="Your prompt")
    with gr.Row():
        with gr.Column():
            negative_prompt = gr.Textbox(label="Negative prompt")
            steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                      info="In many steps will the denoiser denoise the image?")
            guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                      info="Controls how much the text prompt influences the result")
            width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
            height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
            btn = gr.Button("Submit")
        with gr.Column():
            output = gr.Image(label="Result")

    btn.click(fn=get_completion, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])
gr.close_all()
demo.launch(share=True)

In [ ]:
demo.close()

#### scale

- To choose how much relative width to give to each column, set the `scale` parameter of each `gr.Column()`.  
- If one column has `scale=4` and the second column has `scale=1`, then the first column takes up 4/5 of the total width, and the second column takes up 1/5 of the total width.

#### gr.Accordion()
- The `gr.Accordion()` can show/hide  the app options with a mouse click.
- Set `open=True` to show the contents of the Accordion by default, or `False` to hide it by default.

In [ ]:
with gr.Blocks() as demo:
    gr.Markdown("# Image Generation with Stable Diffusion")
    with gr.Row():
        with gr.Column(scale=4):
            prompt = gr.Textbox(label="Your prompt") #Give prompt some real estate
        with gr.Column(scale=1, min_width=50):
            btn = gr.Button("Submit") #Submit button side by side!
    with gr.Accordion("Advanced options", open=False): #Let's hide the advanced options!
            negative_prompt = gr.Textbox(label="Negative prompt")
            with gr.Row():
                with gr.Column():
                    steps = gr.Slider(label="Inference Steps", minimum=1, maximum=100, value=25,
                      info="In many steps will the denoiser denoise the image?")
                    guidance = gr.Slider(label="Guidance Scale", minimum=1, maximum=20, value=7,
                      info="Controls how much the text prompt influences the result")
                with gr.Column():
                    width = gr.Slider(label="Width", minimum=64, maximum=512, step=64, value=512)
                    height = gr.Slider(label="Height", minimum=64, maximum=512, step=64, value=512)
    output = gr.Image(label="Result") #Move the output up too
            
    btn.click(fn=get_completion, inputs=[prompt,negative_prompt,steps,guidance,width,height], outputs=[output])

gr.close_all()
demo.launch(share=True)

In [ ]:
gr.close_all()